In [1]:
import os
import azureml.core
from azureml.core import Workspace, Experiment, Datastore, Dataset
from azureml.widgets import RunDetails

Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.scriptrun = azureml.core.script_run:ScriptRun._from_run_dto with exception (pywin32 228 (c:\users\souvik\miniconda3\envs\azure\lib\site-packages), Requirement.parse('pywin32==227; sys_platform == "win32"'), {'docker'}).


In [2]:
azureml.core.VERSION

'1.1.5'

In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
amlcompute_cluster_name = "cpu-cluster"
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)
aml_compute.wait_for_completion(show_output=True)


In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
RandomParameterSampling({
    "C": uniform(0.8, 1.2)
    "max_iter": choie(100,125, 150)
    })

# Specify a Policy
policy = BanditPolicy(evaluation_interval= 4, slack_factor = 0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
sk_est = SKLearn(soure_directory= "./training",
			compute_target= aml_compute,
			entry_script= "train.py",
			)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(ps,
										estimator= sk_est,
										primary_metric_name= "Accuracy",
										primary_metric_goal= "MAXIMIZE",
										max_duration_minutes= 30,
										policy= policy
										)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

exp_run_obj= exp.submit(hyperdrive_config)

In [ ]:
import joblib

# Get your best run and save the model from that run.

best_run= exp_run_obj.get_best_run_by_primary_metric()
best_run_results= best_run.get_metrics()
print("Best Accuracy= {}".format(best_run_results['Accuracy']))
model= best_run.register(model_name= "scikit_model", 
						model_path= 'outputs/scikit_model.joblib')

In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset
import pandas as pd

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data_url= "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
az_dataset= Dataset.Tabular.from_delimited_files(path= data_url)
print(az_dataset.to_pandas_dataframe().head())

PandasImportError: Could not import pandas. Ensure a compatible version is installed by running: pip install azureml-dataprep[pandas]


DatasetExecutionError: Could not import pandas. Ensure a compatible version is installed by running: pip install azureml-dataprep[pandas]

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
X_df, y_df= clean_data(az_dataset)
X_df["y"]= y_df["y"]

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    X_valid= ,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###